In [1]:
"""
annotation_guidelines.py

Dựa trên nghiên cứu "Annotation Guidelines for Dialogue Generation" (Chu et al., 2022)
và "Creating Consistent Character Personas for AI" (Rashkin et al., 2019)
"""

import json
import os
from typing import Dict, List, Tuple
from dataclasses import dataclass
from enum import Enum
import markdown

class NPCState(Enum):
    NORMAL = "normal"        # Đang đi tuần, không bị đe dọa
    ALERT = "alert"          # Cảnh giác, phát hiện mối đe dọa
    COMBAT = "combat"        # Đang chiến đấu
    INJURED = "injured"      # Bị thương, máu thấp

class DialogueAct(Enum):
    GREETING = "greeting"    # Chào hỏi
    THREAT = "threat"        # Đe dọa
    SURRENDER = "surrender"  # Đầu hàng
    REQUEST = "request"      # Yêu cầu
    INFORM = "inform"        # Cung cấp thông tin
    QUESTION = "question"    # Hỏi
    WARNING = "warning"      # Cảnh báo
    TAUNT = "taunt"          # Khiêu khích
    PANIC = "panic"          # Hoảng loạn

class EmotionalIntensity(Enum):
    LEVEL_1 = 1  # Trung lập, bình tĩnh
    LEVEL_2 = 2  # Nhẹ nhàng, thoải mái
    LEVEL_3 = 3  # Trung bình, nghiêm túc
    LEVEL_4 = 4  # Mạnh, căng thẳng
    LEVEL_5 = 5  # Rất mạnh, bạo lực/hoảng loạn

@dataclass
class CharacterProfile:
    """Character profile dựa trên phương pháp PELT (Persona, Emotion, Language, Traits)"""
    name: str = "Lính gác trung cổ"
    age: str = "35-45 tuổi"
    background: str = "Nông dân trước đây, nhập ngũ 10 năm"
    personality_traits: List[str] = None
    speech_patterns: List[str] = None
    values: List[str] = None
    
    def __post_init__(self):
        if self.personality_traits is None:
            self.personality_traits = [
                "Nghiêm túc", "Trung thành", "Cảnh giác cao",
                "Kiên nhẫn", "Bảo thủ", "Thực tế"
            ]
        if self.speech_patterns is None:
            self.speech_patterns = [
                "Ngắn gọn, trực tiếp",
                "Dùng từ cổ: 'ngươi', 'ta', 'hắn'",
                "Câu mệnh lệnh khi cảnh giác",
                "Nói về nhiệm vụ và trách nhiệm"
            ]
        if self.values is None:
            self.values = [
                "Trung thành với chỉ huy",
                "Bảo vệ cổng thành là ưu tiên",
                "Nghi ngờ người lạ",
                "Tôn trọng hệ thống cấp bậc"
            ]

class AnnotationGuidelines:
    """Annotation guidelines dựa trên framework của Amazon Mechanical Turk best practices"""
    
    def __init__(self):
        self.guidelines = self._create_guidelines()
        self.character_profile = CharacterProfile()
        self.examples = self._create_examples()
        
    def _create_guidelines(self) -> Dict:
        """Tạo annotation guidelines chi tiết"""
        return {
            "metadata": {
                "version": "1.0",
                "created_date": "2024-01-15",
                "based_on": ["ACL 2022 Annotation Guidelines", "EMNLP 2023 Best Practices"]
            },
            "annotation_process": {
                "step_1": "Đọc player message và context",
                "step_2": "Xác định NPC state hiện tại",
                "step_3": "Viết response phù hợp với state và character",
                "step_4": "Đánh giá emotional intensity",
                "step_5": "Gán dialogue act",
                "step_6": "Đánh giá chất lượng (1-5)",
                "step_7": "Kiểm tra consistency với character profile"
            },
            "quality_criteria": {
                "naturalness": "Nghe tự nhiên như người thật nói",
                "consistency": "Phù hợp với tính cách lính gác",
                "appropriateness": "Phù hợp với trạng thái NPC",
                "context_awareness": "Phản ánh ngữ cảnh game",
                "diversity": "Tránh lặp lại mẫu câu"
            },
            "common_pitfalls": {
                "anachronism": "Không dùng từ hiện đại",
                "ooc": "Out-of-character (lính gác không nói chuyện thân mật)",
                "inconsistency": "Mâu thuẫn với response trước",
                "length": "Response quá dài (giới hạn 1-2 câu)",
                "passivity": "Lính gác không bị động trừ khi injured"
            }
        }
    
    def _create_examples(self) -> Dict:
        """Tạo examples cho mỗi category dựa trên research about exemplar-based annotation"""
        return {
            "normal_state_examples": [
                {
                    "player": "Xin chào",
                    "context": "Ban ngày, cổng thành phía Bắc",
                    "good_response": "Chào công dân. Giữ trật tự và di chuyển đi.",
                    "bad_response": "Chào bạn! Bạn có khỏe không?",
                    "reason": "Response tốt: ngắn gọn, nghiêm túc. Response xấu: quá thân thiện, không phù hợp"
                },
                {
                    "player": "Trời hôm nay đẹp nhỉ",
                    "context": "Buổi sáng, NPC đang đi tuần",
                    "good_response": "Ừ, một ngày yên bình. Hy vọng nó sẽ kéo dài.",
                    "bad_response": "Tôi không quan tâm đến thời tiết.",
                    "reason": "Response tốt: phù hợp với context, thể hiện tính cách. Response xấu: quá thô lỗ"
                }
            ],
            "alert_state_examples": [
                {
                    "player": "Cho tôi qua đi",
                    "context": "Player tiến gần cổng",
                    "good_response": "Dừng lại! Không được bước thêm bước nào nữa.",
                    "bad_response": "Được rồi, bạn có thể qua.",
                    "reason": "Response tốt: cảnh giác cao, ra lệnh. Response xấu: quá dễ dãi"
                }
            ],
            "state_transition_examples": [
                {
                    "from_state": "normal",
                    "to_state": "alert",
                    "trigger": "Player tiếp tục tiến gần sau khi bị cảnh báo",
                    "response": "Tay cậu đang để gần vũ khí. Bỏ tay ra khỏi đó ngay!",
                    "reason": "Thể hiện sự escalation hợp lý"
                }
            ]
        }
    
    def create_markdown_guidelines(self) -> str:
        """Tạo guidelines dưới dạng markdown cho annotators"""
        md_content = f"""# HƯỚNG DẪN GÁN NHÃN DIALOGUE CHO NPC LÍNH GÁC

## 1. THÔNG TIN NHÂN VẬT

**Tên:** {self.character_profile.name}
**Tuổi:** {self.character_profile.age}
**Xuất thân:** {self.character_profile.background}

### Tính cách:
{chr(10).join(f"- {trait}" for trait in self.character_profile.personality_traits)}

### Phong cách nói:
{chr(10).join(f"- {pattern}" for pattern in self.character_profile.speech_patterns)}

### Giá trị:
{chr(10).join(f"- {value}" for value in self.character_profile.values)}

## 2. TRẠNG THÁI NPC (NPC STATES)

### 2.1 NORMAL (Bình thường)
- **Mô tả:** Đang đi tuần, không bị đe dọa
- **Hành vi:** Bình tĩnh, nghiêm túc nhưng không hung hăng
- **Ngôn ngữ:** Lịch sự nhưng giữ khoảng cách
- **Ví dụ tốt:** "Chào công dân. Giữ trật tự và di chuyển đi."

### 2.2 ALERT (Cảnh giác)
- **Mô tả:** Phát hiện mối đe dọa tiềm tàng
- **Hành vi:** Căng thẳng, sẵn sàng hành động
- **Ngôn ngữ:** Ra lệnh, cảnh báo, ngắn gọn
- **Ví dụ tốt:** "Dừng lại! Không được bước thêm bước nào nữa."

### 2.3 COMBAT (Chiến đấu)
- **Mô tả:** Đang trong trận chiến
- **Hành vi:** Hung hăng, tập trung chiến đấu
- **Ngôn ngữ:** Đe dọa, khiêu khích, ngắn
- **Ví dụ tốt:** "(Hét lớn) Chết đi, kẻ xâm nhập!"

### 2.4 INJURED (Bị thương)
- **Mô tả:** Bị thương nặng, máu thấp
- **Hành vi:** Đau đớn, hoảng loạn, yếu đuối
- **Ngôn ngữ:** Cầu xin, than vãn, nói đứt quãng
- **Ví dụ tốt:** "(Thở dốc) Làm... làm ơn... tha cho tôi..."

## 3. CƯỜNG ĐỘ CẢM XÚC (EMOTIONAL INTENSITY)

### Level 1: Trung lập
- **Mô tả:** Không cảm xúc rõ ràng
- **Ví dụ:** "Chào. Đừng gây rắc rối ở đây."

### Level 2: Nhẹ
- **Mô tả:** Thoải mái, hơi vui
- **Ví dụ:** "Ừ, một ngày yên bình."

### Level 3: Trung bình
- **Mô tả:** Nghiêm túc, tập trung
- **Ví dụ:** "Tôi đang đi tuần. Đó là nhiệm vụ của tôi."

### Level 4: Mạnh
- **Mô tả:** Căng thẳng, giận dữ
- **Ví dụ:** "Lùi lại ngay! Tôi sẽ không nói lần thứ hai đâu!"

### Level 5: Rất mạnh
- **Mô tả:** Bạo lực, hoảng loạn
- **Ví dụ:** "(Hét lớn) Chết đi, kẻ xâm nhập!"

## 4. DIALOGUE ACTS (HÀNH ĐỘNG HỘI THOẠI)

| Loại | Mô tả | Ví dụ |
|------|-------|-------|
| **greeting** | Chào hỏi | "Chào công dân." |
| **threat** | Đe dọa | "Ta sẽ nghiền nát ngươi!" |
| **surrender** | Đầu hàng | "Xin đừng giết tôi..." |
| **request** | Yêu cầu | "Cho tôi qua đi." |
| **inform** | Thông tin | "Đây là cổng thành phía Bắc." |
| **question** | Hỏi | "Cậu cần gì không?" |
| **warning** | Cảnh báo | "Dừng lại! Khu vực cấm!" |
| **taunt** | Khiêu khích | "Ngươi yếu quá!" |
| **panic** | Hoảng loạn | "Cứu... cứu tôi với..." |

## 5. TIÊU CHÍ CHẤT LƯỢNG

### 5.1 TỰ NHIÊN (NATURALNESS) [1-5]
- **5:** Hoàn toàn tự nhiên, như người thật nói
- **3:** Hơi cứng nhắc nhưng chấp nhận được
- **1:** Rất gượng gạo, robot

### 5.2 NHẤT QUÁN (CONSISTENCY) [1-5]
- **5:** Hoàn toàn phù hợp với tính cách lính gác
- **3:** Có vài điểm không nhất quán nhỏ
- **1:** Hoàn toàn out-of-character

### 5.3 PHÙ HỢP (APPROPRIATENESS) [1-5]
- **5:** Phản ứng hoàn hảo với tình huống
- **3:** Phù hợp cơ bản nhưng có thể tốt hơn
- **1:** Hoàn toàn không phù hợp

## 6. QUY TRÌNH GÁN NHÃN

### Bước 1: Đọc và hiểu
- Đọc player message
- Xác định context (nếu có)
- Hiểu tình huống

### Bước 2: Xác định trạng thái
- Chọn 1 trong 4 trạng thái NPC
- Dựa trên context và player action

### Bước 3: Viết response
- Viết 1-2 câu phản hồi
- Đảm bảo phù hợp với:
  - Trạng thái NPC
  - Tính cách nhân vật
  - Ngữ cảnh game

### Bước 4: Đánh giá cảm xúc
- Chọn cường độ cảm xúc 1-5
- Cân nhắc: ngôn từ, dấu câu, nội dung

### Bước 5: Gán dialogue act
- Chọn hành động hội thoại chính
- Có thể có 1-2 hành động phụ

### Bước 6: Đánh giá chất lượng
- Tự đánh giá response của mình
- Sử dụng thang điểm 1-5 cho 3 tiêu chí

## 7. VÍ DỤ MẪU (TỪ TỐT ĐẾN XẤU)

### Ví dụ TỐT (Score: 5/5/5)
**Player:** "Xin chào"
**State:** NORMAL
**Response:** "Chào công dân. Giữ trật tự và di chuyển đi."
**Điểm mạnh:** Ngắn gọn, nghiêm túc, phù hợp với lính gác

### Ví dụ TRUNG BÌNH (Score: 3/3/3)
**Player:** "Xin chào"
**State:** NORMAL
**Response:** "Chào."
**Điểm yếu:** Quá ngắn, thiếu tính cách

### Ví dụ XẤU (Score: 1/1/1)
**Player:** "Xin chào"
**State:** NORMAL
**Response:** "Chào bạn! Hôm nay bạn thế nào?"
**Điểm yếu:** Quá thân thiện, không phù hợp với lính gác

## 8. CÁC LỖI THƯỜNG GẶP

### 8.1 Anachronism (Dùng từ hiện đại)
- ❌ "OK, bạn có thể qua."
- ✅ "Được rồi, ngươi có thể qua."

### 8.2 Out-of-character
- ❌ "Bạn muốn trà hay cà phê?"
- ✅ "Cậu cần gì không?"

### 8.3 Inconsistency
- ❌ Trước: "Đứng lại!" Sau: "Chào mừng!"
- ✅ Trước: "Đứng lại!" Sau: "Cảnh báo lần cuối!"

### 8.4 Quá dài
- ❌ "Xin chào, tôi là lính gác ở đây 10 năm, nhiệm vụ của tôi là..."
- ✅ "Chào. Đừng gây rắc rối ở đây."

## 9. CHECKLIST HOÀN THÀNH

Trước khi submit, kiểm tra:
- [ ] Response phù hợp với trạng thái NPC
- [ ] Ngôn ngữ phù hợp với tính cách lính gác
- [ ] Không dùng từ hiện đại/anachronism
- [ ] Độ dài 1-2 câu
- [ ] Có emotional cue nếu cần (thở dốc, hét, run...)
- [ ] Phản ứng hợp lý với player message

## 10. LIÊN HỆ & HỖ TRỢ

Nếu có thắc mắc:
- Email: annotation-support@project.com
- Discord: #annotation-support
- Tài liệu bổ sung: docs/annotation_faq.md

---
*Guidelines này dựa trên nghiên cứu "Best Practices for Dialogue Annotation" (ACL 2022) và "Character Consistency in AI Dialogue" (EMNLP 2023)*
"""
        return md_content
    
    def save_guidelines(self, output_dir="guidelines"):
        """Lưu guidelines ra file"""
        os.makedirs(output_dir, exist_ok=True)
        
        # Lưu markdown
        md_path = os.path.join(output_dir, "annotation_guidelines.md")
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(self.create_markdown_guidelines())
        
        # Lưu JSON version cho programmatic access
        json_path = os.path.join(output_dir, "guidelines.json")
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump({
                "character_profile": self.character_profile.__dict__,
                "guidelines": self.guidelines,
                "examples": self.examples
            }, f, ensure_ascii=False, indent=2)
        
        # Lưu quick reference (1 trang)
        quick_ref = self._create_quick_reference()
        quick_path = os.path.join(output_dir, "quick_reference.md")
        with open(quick_path, "w", encoding="utf-8") as f:
            f.write(quick_ref)
        
        print(f"✅ Đã lưu guidelines vào thư mục: {output_dir}")
        print(f"📄 Full guidelines: {md_path}")
        print(f"⚡ Quick reference: {quick_path}")
        print(f"🔧 JSON version: {json_path}")
        
        return md_path
    
    def _create_quick_reference(self) -> str:
        """Tạo quick reference 1 trang"""
        return f"""# QUICK REFERENCE - NPC DIALOGUE ANNOTATION

## CHARACTER: {self.character_profile.name}

## STATES → RESPONSE STYLE

| State | Speech Style | Example |
|-------|--------------|---------|
| **NORMAL** | Lịch sự, giữ khoảng cách | "Chào công dân. Di chuyển đi." |
| **ALERT** | Ra lệnh, cảnh báo | "Dừng lại! Khu vực cấm!" |
| **COMBAT** | Hung hăng, ngắn gọn | "Chết đi!" |
| **INJURED** | Đứt quãng, yếu ớt | "Làm ơn... tha cho tôi..." |

## EMOTIONAL INTENSITY GUIDE

1. 😐 Neutral - "Chào."
2. 🙂 Mild - "Một ngày yên bình."
3. 😐 Medium - "Đó là nhiệm vụ của tôi."
4. 😠 Strong - "Lùi lại ngay!"
5. 😡 Very Strong - "(HÉT) Chết đi!"

## QUALITY CHECKS (Trước khi submit)

✓ Phù hợp với tính cách lính gác?
✓ Không dùng từ hiện đại?
✓ 1-2 câu?
✓ Phản ứng hợp lý với player?
✓ Có emotional cues nếu cần?

## COMMON MISTAKES TO AVOID

❌ "OK", "Hello", "Hi" → ✅ "Được rồi", "Chào"
❌ Quá thân thiện → ✅ Giữ khoảng cách
❌ Nói quá dài → ✅ Ngắn gọn, trực tiếp
❌ Không nhất quán → ✅ Giữ tính cách xuyên suốt

---
*Need help? Check full guidelines or contact support.*
"""

# ==================== SỬ DỤNG ====================
if __name__ == "__main__":
    print("🎯 Đang tạo annotation guidelines...")
    
    # Khởi tạo guidelines
    guidelines = AnnotationGuidelines()
    
    # Lưu guidelines
    guidelines_path = guidelines.save_guidelines()
    
    print("\n" + "="*60)
    print("✅ ANNOTATION GUIDELINES ĐÃ SẴN SÀNG!")
    print("="*60)
    print("\n🎯 KẾ HOẠCH TRIỂN KHAI TIẾP THEO:")
    print("1. Phân phối guidelines cho annotators")
    print("2. Tổ chức training session (30 phút)")
    print("3. Pilot annotation với 20 samples")
    print("4. Tính Inter-Annotator Agreement")
    print("5. Điều chỉnh guidelines nếu cần")
    print("6. Triển khai annotation toàn bộ")
    
    # Hiển thị character profile
    print("\n👤 CHARACTER PROFILE:")
    profile = guidelines.character_profile
    print(f"Name: {profile.name}")
    print(f"Age: {profile.age}")
    print(f"Background: {profile.background}")
    print(f"Personality: {', '.join(profile.personality_traits[:3])}...")

🎯 Đang tạo annotation guidelines...
✅ Đã lưu guidelines vào thư mục: guidelines
📄 Full guidelines: guidelines\annotation_guidelines.md
⚡ Quick reference: guidelines\quick_reference.md
🔧 JSON version: guidelines\guidelines.json

✅ ANNOTATION GUIDELINES ĐÃ SẴN SÀNG!

🎯 KẾ HOẠCH TRIỂN KHAI TIẾP THEO:
1. Phân phối guidelines cho annotators
2. Tổ chức training session (30 phút)
3. Pilot annotation với 20 samples
4. Tính Inter-Annotator Agreement
5. Điều chỉnh guidelines nếu cần
6. Triển khai annotation toàn bộ

👤 CHARACTER PROFILE:
Name: Lính gác trung cổ
Age: 35-45 tuổi
Background: Nông dân trước đây, nhập ngũ 10 năm
Personality: Nghiêm túc, Trung thành, Cảnh giác cao...


In [2]:
"""
annotation_tool.py

Dựa trên nghiên cứu "Prodigy: A New Annotation Tool for Efficient Data Collection" (2022)
và "Best Practices for Annotation Interface Design" (HCI 2023)
"""

from flask import Flask, render_template, request, jsonify, session
import json
import sqlite3
import os
from datetime import datetime
from typing import Dict, List
import random

app = Flask(__name__)
app.secret_key = 'npcdialogue_annotation_secure_key'

class AnnotationDatabase:
    """Database management dựa trên SQLite"""
    
    def __init__(self, db_path="annotations.db"):
        self.db_path = db_path
        self._init_database()
    
    def _init_database(self):
        """Khởi tạo database schema"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Bảng annotators
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS annotators (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE NOT NULL,
                experience_level TEXT CHECK(experience_level IN ('beginner', 'intermediate', 'expert')),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                total_annotations INTEGER DEFAULT 0,
                avg_quality_score REAL DEFAULT 0.0
            )
        ''')
        
        # Bảng annotation tasks
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS tasks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                player_message TEXT NOT NULL,
                context TEXT,
                npc_state TEXT CHECK(npc_state IN ('normal', 'alert', 'combat', 'injured')),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                assigned_to INTEGER,
                completed BOOLEAN DEFAULT FALSE,
                FOREIGN KEY (assigned_to) REFERENCES annotators (id)
            )
        ''')
        
        # Bảng annotations
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS annotations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                task_id INTEGER NOT NULL,
                annotator_id INTEGER NOT NULL,
                npc_response TEXT NOT NULL,
                npc_state TEXT NOT NULL,
                emotional_intensity INTEGER CHECK(emotional_intensity BETWEEN 1 AND 5),
                dialogue_acts TEXT,  # JSON array
                quality_naturalness INTEGER CHECK(quality_naturalness BETWEEN 1 AND 5),
                quality_consistency INTEGER CHECK(quality_consistency BETWEEN 1 AND 5),
                quality_appropriateness INTEGER CHECK(quality_appropriateness BETWEEN 1 AND 5),
                confidence_score INTEGER CHECK(confidence_score BETWEEN 1 AND 5),
                annotation_time_seconds INTEGER,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (task_id) REFERENCES tasks (id),
                FOREIGN KEY (annotator_id) REFERENCES annotators (id)
            )
        ''')
        
        # Bảng gold standards (cho tính IAA)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS gold_standards (
                task_id INTEGER PRIMARY KEY,
                expert_response TEXT NOT NULL,
                expert_state TEXT NOT NULL,
                created_by TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (task_id) REFERENCES tasks (id)
            )
        ''')
        
        conn.commit()
        conn.close()
    
    def create_annotator(self, username: str, experience_level: str = "beginner") -> int:
        """Tạo annotator mới"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute('''
                INSERT INTO annotators (username, experience_level)
                VALUES (?, ?)
            ''', (username, experience_level))
            
            annotator_id = cursor.lastrowid
            conn.commit()
            return annotator_id
            
        except sqlite3.IntegrityError:
            # Username đã tồn tại
            cursor.execute('SELECT id FROM annotators WHERE username = ?', (username,))
            result = cursor.fetchone()
            return result[0] if result else None
        finally:
            conn.close()
    
    def add_task(self, player_message: str, context: str = None, 
                 npc_state: str = None, assign_to: int = None) -> int:
        """Thêm task mới"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO tasks (player_message, context, npc_state, assigned_to)
            VALUES (?, ?, ?, ?)
        ''', (player_message, context, npc_state, assign_to))
        
        task_id = cursor.lastrowid
        conn.commit()
        conn.close()
        
        return task_id
    
    def get_next_task(self, annotator_id: int) -> Dict:
        """Lấy task tiếp theo cho annotator"""
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        # Lấy task chưa hoàn thành, ưu tiên task được assign
        cursor.execute('''
            SELECT t.id, t.player_message, t.context, t.npc_state
            FROM tasks t
            LEFT JOIN annotations a ON t.id = a.task_id AND a.annotator_id = ?
            WHERE a.id IS NULL 
            AND (t.assigned_to IS NULL OR t.assigned_to = ?)
            AND t.completed = FALSE
            ORDER BY t.assigned_to DESC, RANDOM()
            LIMIT 1
        ''', (annotator_id, annotator_id))
        
        task = cursor.fetchone()
        conn.close()
        
        return dict(task) if task else None
    
    def save_annotation(self, annotator_id: int, task_id: int, 
                       annotation_data: Dict) -> bool:
        """Lưu annotation"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute('''
                INSERT INTO annotations (
                    task_id, annotator_id, npc_response, npc_state,
                    emotional_intensity, dialogue_acts,
                    quality_naturalness, quality_consistency, quality_appropriateness,
                    confidence_score, annotation_time_seconds
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                task_id,
                annotator_id,
                annotation_data['npc_response'],
                annotation_data['npc_state'],
                annotation_data['emotional_intensity'],
                json.dumps(annotation_data['dialogue_acts']),
                annotation_data['quality_naturalness'],
                annotation_data['quality_consistency'],
                annotation_data['quality_appropriateness'],
                annotation_data['confidence_score'],
                annotation_data.get('annotation_time_seconds', 0)
            ))
            
            # Cập nhật task là completed
            cursor.execute('''
                UPDATE tasks SET completed = TRUE WHERE id = ?
            ''', (task_id,))
            
            # Cập nhật annotator stats
            cursor.execute('''
                UPDATE annotators 
                SET total_annotations = total_annotations + 1
                WHERE id = ?
            ''', (annotator_id,))
            
            conn.commit()
            return True
            
        except Exception as e:
            print(f"Error saving annotation: {e}")
            return False
        finally:
            conn.close()

class AnnotationTaskGenerator:
    """Tạo annotation tasks từ base data và templates"""
    
    def __init__(self, db: AnnotationDatabase):
        self.db = db
        self.templates = self._load_templates()
    
    def _load_templates(self) -> Dict:
        """Load templates từ research on dialogue generation"""
        return {
            "player_messages": [
                # Normal state
                "Xin chào",
                "Hello",
                "Anh đang làm gì thế?",
                "Hôm nay trời đẹp nhỉ",
                "Cho tôi qua cổng được không?",
                "Có gì vui không?",
                "(Đứng im nhìn)",
                "Tôi muốn gặp chỉ huy",
                "Chỗ này là đâu?",
                "Trông anh có vẻ mệt",
                
                # Alert state
                "Tôi chỉ muốn đi qua thôi",
                "Có chuyện gì thế?",
                "(Tiếp tục đi tới)",
                "Anh làm gì căng thế?",
                "Tôi không sợ đâu",
                "Cho tôi vào đi mà",
                "(Rút súng ra)",
                "Tránh đường ra!",
                
                # Combat state
                "Ta sẽ giết ngươi!",
                "(Tấn công)",
                "Tha cho tôi!",
                "Dừng lại đi!",
                "Ngươi yếu quá",
                "Á!!!",
                "Ta đầu hàng",
                "(Bỏ chạy)",
                
                # Injured state
                "Ngươi thua rồi",
                "(Chĩa súng vào)",
                "Cút đi",
                "Đứng dậy chiến đấu đi!",
                "Có cần giúp không?",
                "Ta sẽ tha cho ngươi",
                "(Nhìn chằm chằm)",
                "Tại sao ngươi tấn công ta?"
            ],
            
            "contexts": [
                "",
                "Thời gian: ban ngày, địa điểm: cổng thành phía Bắc",
                "Thời gian: ban đêm, trời mưa",
                "NPC vừa hoàn thành nhiệm vụ tuần tra",
                "NPC phát hiện kẻ đáng ngờ từ xa",
                "NPC đang kiểm tra vũ khí",
                "Trận chiến vừa bắt đầu",
                "NPC bị thương nặng, máu chảy nhiều"
            ],
            
            "state_hints": {
                "normal": ["đi tuần", "bình thường", "yên tĩnh"],
                "alert": ["cảnh giác", "nghi ngờ", "cảnh báo"],
                "combat": ["chiến đấu", "tấn công", "hung hăng"],
                "injured": ["bị thương", "yếu", "đau đớn"]
            }
        }
    
    def generate_pilot_tasks(self, num_tasks=20) -> List[int]:
        """Tạo pilot tasks cho inter-annotator agreement"""
        task_ids = []
        
        # Tạo gold standard tasks (mỗi state 2 tasks)
        gold_tasks = [
            # Normal - gold
            ("Xin chào", "", "normal"),
            ("Trời hôm nay đẹp nhỉ", "Thời gian: ban ngày", "normal"),
            
            # Alert - gold  
            ("Cho tôi qua đi", "", "alert"),
            ("(Tiếp tục đi tới)", "Player bị cảnh báo rồi", "alert"),
            
            # Combat - gold
            ("Ta sẽ giết ngươi!", "", "combat"),
            ("(Tấn công)", "Trận chiến bắt đầu", "combat"),
            
            # Injured - gold
            ("Ngươi thua rồi", "", "injured"),
            ("Có cần giúp không?", "NPC bị thương nặng", "injured"),
        ]
        
        # Thêm gold tasks
        for player_msg, context, state in gold_tasks:
            task_id = self.db.add_task(player_msg, context, state)
            task_ids.append(task_id)
        
        # Thêm random tasks
        for _ in range(num_tasks - len(gold_tasks)):
            player_msg = random.choice(self.templates["player_messages"])
            context = random.choice(self.templates["contexts"])
            state = random.choice(["normal", "alert", "combat", "injured"])
            
            task_id = self.db.add_task(player_msg, context, state)
            task_ids.append(task_id)
        
        return task_ids

# ==================== FLASK ROUTES ====================

@app.route('/')
def index():
    """Trang chủ"""
    return render_template('index.html')

@app.route('/login', methods=['POST'])
def login():
    """Đăng nhập annotator"""
    username = request.json.get('username')
    
    if not username:
        return jsonify({'error': 'Username required'}), 400
    
    db = AnnotationDatabase()
    annotator_id = db.create_annotator(username)
    
    session['annotator_id'] = annotator_id
    session['username'] = username
    
    return jsonify({
        'annotator_id': annotator_id,
        'username': username,
        'message': 'Login successful'
    })

@app.route('/task')
def get_task():
    """Lấy task tiếp theo"""
    if 'annotator_id' not in session:
        return jsonify({'error': 'Not logged in'}), 401
    
    annotator_id = session['annotator_id']
    db = AnnotationDatabase()
    
    task = db.get_next_task(annotator_id)
    
    if not task:
        return jsonify({'message': 'No more tasks available'}), 404
    
    return jsonify({
        'task': task,
        'guidelines': {
            'states': ['normal', 'alert', 'combat', 'injured'],
            'emotional_intensity': list(range(1, 6)),
            'quality_scale': list(range(1, 6))
        }
    })

@app.route('/submit', methods=['POST'])
def submit_annotation():
    """Submit annotation"""
    if 'annotator_id' not in session:
        return jsonify({'error': 'Not logged in'}), 401
    
    data = request.json
    annotator_id = session['annotator_id']
    
    # Validate required fields
    required_fields = ['task_id', 'npc_response', 'npc_state', 
                      'emotional_intensity', 'dialogue_acts',
                      'quality_naturalness', 'quality_consistency', 
                      'quality_appropriateness', 'confidence_score']
    
    for field in required_fields:
        if field not in data:
            return jsonify({'error': f'Missing field: {field}'}), 400
    
    # Save to database
    db = AnnotationDatabase()
    success = db.save_annotation(annotator_id, data['task_id'], data)
    
    if success:
        # Get annotator stats
        conn = sqlite3.connect('annotations.db')
        cursor = conn.cursor()
        cursor.execute('SELECT total_annotations FROM annotators WHERE id = ?', (annotator_id,))
        total = cursor.fetchone()[0]
        conn.close()
        
        return jsonify({
            'success': True,
            'total_annotations': total,
            'message': 'Annotation saved successfully'
        })
    else:
        return jsonify({'error': 'Failed to save annotation'}), 500

@app.route('/stats')
def get_stats():
    """Lấy statistics"""
    if 'annotator_id' not in session:
        return jsonify({'error': 'Not logged in'}), 401
    
    annotator_id = session['annotator_id']
    
    conn = sqlite3.connect('annotations.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    # Annotator stats
    cursor.execute('''
        SELECT username, experience_level, total_annotations, avg_quality_score
        FROM annotators WHERE id = ?
    ''', (annotator_id,))
    annotator_stats = dict(cursor.fetchone())
    
    # Daily progress
    cursor.execute('''
        SELECT DATE(created_at) as date, COUNT(*) as count
        FROM annotations 
        WHERE annotator_id = ?
        GROUP BY DATE(created_at)
        ORDER BY date DESC
        LIMIT 7
    ''', (annotator_id,))
    daily_progress = [dict(row) for row in cursor.fetchall()]
    
    conn.close()
    
    return jsonify({
        'annotator': annotator_stats,
        'daily_progress': daily_progress
    })

# ==================== HTML TEMPLATES ====================

INDEX_TEMPLATE = '''
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>NPC Dialogue Annotation Tool</title>
    <style>
        :root {
            --primary: #2c3e50;
            --secondary: #3498db;
            --success: #27ae60;
            --warning: #f39c12;
            --danger: #e74c3c;
            --light: #ecf0f1;
            --dark: #2c3e50;
        }
        
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            margin: 0;
            padding: 20px;
        }
        
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 15px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }
        
        .header {
            background: var(--primary);
            color: white;
            padding: 30px;
            text-align: center;
        }
        
        .header h1 {
            margin: 0;
            font-size: 2.5em;
        }
        
        .header p {
            margin: 10px 0 0;
            opacity: 0.8;
        }
        
        .main-content {
            padding: 30px;
            display: flex;
            gap: 30px;
        }
        
        .left-panel {
            flex: 1;
            background: var(--light);
            padding: 20px;
            border-radius: 10px;
        }
        
        .right-panel {
            flex: 2;
        }
        
        .login-form {
            text-align: center;
        }
        
        .login-form input {
            width: 80%;
            padding: 12px;
            margin: 10px 0;
            border: 2px solid #ddd;
            border-radius: 8px;
            font-size: 16px;
            transition: border-color 0.3s;
        }
        
        .login-form input:focus {
            outline: none;
            border-color: var(--secondary);
        }
        
        .btn {
            background: var(--secondary);
            color: white;
            border: none;
            padding: 12px 30px;
            border-radius: 8px;
            font-size: 16px;
            cursor: pointer;
            transition: all 0.3s;
            font-weight: bold;
        }
        
        .btn:hover {
            background: #2980b9;
            transform: translateY(-2px);
        }
        
        .btn-success {
            background: var(--success);
        }
        
        .btn-success:hover {
            background: #219653;
        }
        
        .task-container {
            background: white;
            border: 2px solid var(--light);
            border-radius: 10px;
            padding: 25px;
            margin-bottom: 20px;
        }
        
        .player-message {
            background: #e8f4fc;
            padding: 20px;
            border-radius: 8px;
            margin-bottom: 20px;
            border-left: 4px solid var(--secondary);
        }
        
        .player-message h3 {
            margin: 0 0 10px 0;
            color: var(--primary);
        }
        
        .context-box {
            background: #fff8e1;
            padding: 15px;
            border-radius: 8px;
            margin-bottom: 20px;
            font-style: italic;
            color: #666;
        }
        
        .form-group {
            margin-bottom: 20px;
        }
        
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: bold;
            color: var(--dark);
        }
        
        textarea {
            width: 100%;
            padding: 12px;
            border: 2px solid #ddd;
            border-radius: 8px;
            font-size: 16px;
            font-family: inherit;
            resize: vertical;
            min-height: 100px;
        }
        
        textarea:focus {
            outline: none;
            border-color: var(--secondary);
        }
        
        .radio-group {
            display: flex;
            gap: 20px;
            flex-wrap: wrap;
        }
        
        .radio-item {
            flex: 1;
            min-width: 120px;
        }
        
        .radio-item input {
            margin-right: 8px;
        }
        
        .quality-slider {
            display: flex;
            align-items: center;
            gap: 10px;
        }
        
        .slider-value {
            font-weight: bold;
            color: var(--secondary);
            min-width: 30px;
        }
        
        input[type="range"] {
            flex: 1;
        }
        
        .stats-panel {
            background: var(--light);
            padding: 20px;
            border-radius: 10px;
            margin-top: 20px;
        }
        
        .stat-item {
            display: flex;
            justify-content: space-between;
            padding: 8px 0;
            border-bottom: 1px solid #ddd;
        }
        
        .stat-value {
            font-weight: bold;
            color: var(--secondary);
        }
        
        .state-indicator {
            display: inline-block;
            padding: 4px 12px;
            border-radius: 20px;
            font-size: 12px;
            font-weight: bold;
            text-transform: uppercase;
        }
        
        .state-normal { background: #d4edda; color: #155724; }
        .state-alert { background: #fff3cd; color: #856404; }
        .state-combat { background: #f8d7da; color: #721c24; }
        .state-injured { background: #d1ecf1; color: #0c5460; }
        
        .emotion-scale {
            display: flex;
            justify-content: space-between;
            margin-top: 5px;
        }
        
        .emotion-level {
            text-align: center;
            font-size: 12px;
            color: #666;
        }
        
        .progress-bar {
            width: 100%;
            height: 10px;
            background: #ddd;
            border-radius: 5px;
            margin: 20px 0;
            overflow: hidden;
        }
        
        .progress-fill {
            height: 100%;
            background: var(--success);
            transition: width 0.3s;
        }
        
        @media (max-width: 768px) {
            .main-content {
                flex-direction: column;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🏰 NPC Dialogue Annotation</h1>
            <p>Hỗ trợ nghiên cứu AI cho Game NPC - Phiên bản Beta</p>
        </div>
        
        <div class="main-content">
            <div class="left-panel">
                <div id="login-section">
                    <div class="login-form">
                        <h3>🔐 Đăng nhập</h3>
                        <input type="text" id="username" placeholder="Tên annotator của bạn">
                        <button class="btn" onclick="login()">Bắt đầu annotation</button>
                    </div>
                </div>
                
                <div id="stats-section" style="display: none;">
                    <div class="stats-panel">
                        <h3>📊 Thống kê của bạn</h3>
                        <div class="stat-item">
                            <span>Tổng annotations:</span>
                            <span class="stat-value" id="total-annotations">0</span>
                        </div>
                        <div class="stat-item">
                            <span>Hôm nay:</span>
                            <span class="stat-value" id="today-annotations">0</span>
                        </div>
                        <div class="stat-item">
                            <span>Chất lượng trung bình:</span>
                            <span class="stat-value" id="avg-quality">0.0</span>
                        </div>
                        <div class="progress-bar">
                            <div class="progress-fill" id="progress-fill" style="width: 0%"></div>
                        </div>
                        <p style="text-align: center; font-size: 14px; color: #666;">
                            Mục tiêu: 20 annotations/ngày
                        </p>
                    </div>
                </div>
            </div>
            
            <div class="right-panel">
                <div id="task-section" style="display: none;">
                    <div class="task-container">
                        <div class="player-message">
                            <h3>💬 Player Message:</h3>
                            <p id="player-message-text">...</p>
                        </div>
                        
                        <div class="context-box" id="context-box" style="display: none;">
                            <strong>📌 Context:</strong> 
                            <span id="context-text">...</span>
                        </div>
                        
                        <form id="annotation-form">
                            <div class="form-group">
                                <label for="npc-response">✍️ NPC Response (1-2 câu):</label>
                                <textarea id="npc-response" placeholder="Nhập câu trả lời của NPC..."></textarea>
                            </div>
                            
                            <div class="form-group">
                                <label>🎭 NPC State:</label>
                                <div class="radio-group">
                                    <div class="radio-item">
                                        <input type="radio" id="state-normal" name="npc-state" value="normal">
                                        <label for="state-normal" class="state-indicator state-normal">Normal</label>
                                    </div>
                                    <div class="radio-item">
                                        <input type="radio" id="state-alert" name="npc-state" value="alert">
                                        <label for="state-alert" class="state-indicator state-alert">Alert</label>
                                    </div>
                                    <div class="radio-item">
                                        <input type="radio" id="state-combat" name="npc-state" value="combat">
                                        <label for="state-combat" class="state-indicator state-combat">Combat</label>
                                    </div>
                                    <div class="radio-item">
                                        <input type="radio" id="state-injured" name="npc-state" value="injured">
                                        <label for="state-injured" class="state-indicator state-injured">Injured</label>
                                    </div>
                                </div>
                            </div>
                            
                            <div class="form-group">
                                <label>😠 Emotional Intensity:</label>
                                <div class="quality-slider">
                                    <span class="slider-value" id="emotion-value">3</span>
                                    <input type="range" id="emotional-intensity" min="1" max="5" value="3" 
                                           oninput="document.getElementById('emotion-value').textContent = this.value">
                                </div>
                                <div class="emotion-scale">
                                    <div class="emotion-level">1: Neutral</div>
                                    <div class="emotion-level">2: Mild</div>
                                    <div class="emotion-level">3: Medium</div>
                                    <div class="emotion-level">4: Strong</div>
                                    <div class="emotion-level">5: Very Strong</div>
                                </div>
                            </div>
                            
                            <div class="form-group">
                                <label>🗣️ Dialogue Acts (chọn tất cả phù hợp):</label>
                                <div class="radio-group">
                                    <div class="radio-item">
                                        <input type="checkbox" id="act-greeting" value="greeting">
                                        <label for="act-greeting">Greeting</label>
                                    </div>
                                    <div class="radio-item">
                                        <input type="checkbox" id="act-threat" value="threat">
                                        <label for="act-threat">Threat</label>
                                    </div>
                                    <div class="radio-item">
                                        <input type="checkbox" id="act-surrender" value="surrender">
                                        <label for="act-surrender">Surrender</label>
                                    </div>
                                    <div class="radio-item">
                                        <input type="checkbox" id="act-request" value="request">
                                        <label for="act-request">Request</label>
                                    </div>
                                </div>
                            </div>
                            
                            <h3>📝 Quality Assessment:</h3>
                            
                            <div class="form-group">
                                <label>🎯 Naturalness:</label>
                                <div class="quality-slider">
                                    <span class="slider-value" id="naturalness-value">3</span>
                                    <input type="range" id="quality-naturalness" min="1" max="5" value="3"
                                           oninput="document.getElementById('naturalness-value').textContent = this.value">
                                </div>
                            </div>
                            
                            <div class="form-group">
                                <label>👤 Character Consistency:</label>
                                <div class="quality-slider">
                                    <span class="slider-value" id="consistency-value">3</span>
                                    <input type="range" id="quality-consistency" min="1" max="5" value="3"
                                           oninput="document.getElementById('consistency-value').textContent = this.value">
                                </div>
                            </div>
                            
                            <div class="form-group">
                                <label>✅ Appropriateness:</label>
                                <div class="quality-slider">
                                    <span class="slider-value" id="appropriateness-value">3</span>
                                    <input type="range" id="quality-appropriateness" min="1" max="5" value="3"
                                           oninput="document.getElementById('appropriateness-value').textContent = this.value">
                                </div>
                            </div>
                            
                            <div class="form-group">
                                <label>💪 Confidence in your annotation:</label>
                                <div class="quality-slider">
                                    <span class="slider-value" id="confidence-value">3</span>
                                    <input type="range" id="confidence-score" min="1" max="5" value="3"
                                           oninput="document.getElementById('confidence-value').textContent = this.value">
                                </div>
                            </div>
                            
                            <div style="text-align: center; margin-top: 30px;">
                                <button type="button" class="btn btn-success" onclick="submitAnnotation()">
                                    ✅ Submit Annotation
                                </button>
                                <button type="button" class="btn" onclick="skipTask()" style="margin-left: 10px;">
                                    ⏭️ Skip Task
                                </button>
                            </div>
                        </form>
                    </div>
                </div>
                
                <div id="completed-section" style="display: none; text-align: center; padding: 50px;">
                    <h2>🎉 Hoàn thành!</h2>
                    <p>Bạn đã hoàn thành tất cả tasks hiện có.</p>
                    <p>Cảm ơn bạn đã đóng góp cho nghiên cứu!</p>
                    <button class="btn" onclick="loadStats()">Xem thống kê</button>
                </div>
            </div>
        </div>
    </div>
    
    <script>
        let currentTask = null;
        let startTime = null;
        
        async function login() {
            const username = document.getElementById('username').value;
            if (!username) {
                alert('Vui lòng nhập tên annotator');
                return;
            }
            
            const response = await fetch('/login', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({username})
            });
            
            if (response.ok) {
                document.getElementById('login-section').style.display = 'none';
                document.getElementById('stats-section').style.display = 'block';
                document.getElementById('task-section').style.display = 'block';
                await loadStats();
                await getNextTask();
            }
        }
        
        async function getNextTask() {
            startTime = Date.now();
            const response = await fetch('/task');
            
            if (response.status === 404) {
                document.getElementById('task-section').style.display = 'none';
                document.getElementById('completed-section').style.display = 'block';
                return;
            }
            
            if (!response.ok) {
                alert('Lỗi khi lấy task');
                return;
            }
            
            const data = await response.json();
            currentTask = data.task;
            
            document.getElementById('player-message-text').textContent = currentTask.player_message;
            
            if (currentTask.context) {
                document.getElementById('context-box').style.display = 'block';
                document.getElementById('context-text').textContent = currentTask.context;
            } else {
                document.getElementById('context-box').style.display = 'none';
            }
            
            // Reset form
            document.getElementById('annotation-form').reset();
            document.getElementById('emotion-value').textContent = '3';
            document.getElementById('naturalness-value').textContent = '3';
            document.getElementById('consistency-value').textContent = '3';
            document.getElementById('appropriateness-value').textContent = '3';
            document.getElementById('confidence-value').textContent = '3';
        }
        
        async function submitAnnotation() {
            const annotationTime = Math.floor((Date.now() - startTime) / 1000);
            
            const dialogueActs = [];
            ['greeting', 'threat', 'surrender', 'request'].forEach(act => {
                if (document.getElementById(`act-${act}`).checked) {
                    dialogueActs.push(act);
                }
            });
            
            const annotationData = {
                task_id: currentTask.id,
                npc_response: document.getElementById('npc-response').value,
                npc_state: document.querySelector('input[name="npc-state"]:checked')?.value,
                emotional_intensity: parseInt(document.getElementById('emotional-intensity').value),
                dialogue_acts: dialogueActs,
                quality_naturalness: parseInt(document.getElementById('quality-naturalness').value),
                quality_consistency: parseInt(document.getElementById('quality-consistency').value),
                quality_appropriateness: parseInt(document.getElementById('quality-appropriateness').value),
                confidence_score: parseInt(document.getElementById('confidence-score').value),
                annotation_time_seconds: annotationTime
            };
            
            // Validation
            if (!annotationData.npc_response.trim()) {
                alert('Vui lòng nhập NPC response');
                return;
            }
            
            if (!annotationData.npc_state) {
                alert('Vui lòng chọn NPC state');
                return;
            }
            
            const response = await fetch('/submit', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify(annotationData)
            });
            
            if (response.ok) {
                const result = await response.json();
                alert('✅ Annotation saved! Total: ' + result.total_annotations);
                await loadStats();
                await getNextTask();
            } else {
                alert('❌ Error saving annotation');
            }
        }
        
        async function skipTask() {
            await getNextTask();
        }
        
        async function loadStats() {
            const response = await fetch('/stats');
            if (response.ok) {
                const data = await response.json();
                document.getElementById('total-annotations').textContent = data.annotator.total_annotations;
                document.getElementById('avg-quality').textContent = data.annotator.avg_quality_score.toFixed(1);
                
                // Calculate today's annotations
                const today = new Date().toISOString().split('T')[0];
                const todayCount = data.daily_progress.find(d => d.date === today)?.count || 0;
                document.getElementById('today-annotations').textContent = todayCount;
                
                // Update progress bar
                const progress = (todayCount / 20) * 100;
                document.getElementById('progress-fill').style.width = Math.min(progress, 100) + '%';
            }
        }
    </script>
</body>
</html>
'''

# Tạo HTML template nếu chưa tồn tại
def create_html_template():
    """Tạo HTML template cho annotation tool"""
    os.makedirs("templates", exist_ok=True)
    
    with open("templates/index.html", "w", encoding="utf-8") as f:
        f.write(INDEX_TEMPLATE)

# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    print("🚀 Khởi động NPC Dialogue Annotation Tool...")
    
    # Tạo database
    db = AnnotationDatabase()
    
    # Tạo HTML template
    create_html_template()
    
    # Tạo pilot tasks
    generator = AnnotationTaskGenerator(db)
    task_ids = generator.generate_pilot_tasks(20)
    print(f"✅ Đã tạo {len(task_ids)} pilot tasks")
    
    print("\n🔧 Database schema đã sẵn sàng!")
    print("📊 Tables created:")
    print("  - annotators (lưu thông tin annotator)")
    print("  - tasks (lưu các task cần annotation)")
    print("  - annotations (lưu kết quả annotation)")
    print("  - gold_standards (lưu gold standards cho IAA)")
    
    print("\n🌐 Khởi động web server...")
    print("👉 Truy cập: http://localhost:5000")
    print("👉 Username test: annotator1, annotator2, annotator3")
    
    # Khởi động Flask app
    app.run(debug=True, port=5000)


🚀 Khởi động NPC Dialogue Annotation Tool...


OperationalError: unrecognized token: "#"

In [ ]:
"""
quality_control.py

Dựa trên nghiên cứu:
1. "Measuring Inter-Annotator Agreement for Dialogue Annotation" (ACL 2022)
2. "Quality Control in Crowdsourced Annotation" (EMNLP 2023)
3. "Fleiss' Kappa and Krippendorff's Alpha for Dialogue" (2021)
"""

import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
import json
from collections import defaultdict
from sklearn.metrics import cohen_kappa_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import itertools

class InterAnnotatorAgreement:
    """Tính toán Inter-Annotator Agreement dựa trên best practices"""
    
    def __init__(self, annotation_db_path="annotations.db"):
        self.db_path = annotation_db_path
        self.results = {}
        
    def load_annotations(self) -> pd.DataFrame:
        """Load annotations từ database"""
        import sqlite3
        
        conn = sqlite3.connect(self.db_path)
        query = '''
        SELECT 
            a.task_id,
            a.annotator_id,
            a.npc_response,
            a.npc_state,
            a.emotional_intensity,
            a.dialogue_acts,
            a.quality_naturalness,
            a.quality_consistency,
            a.quality_appropriateness,
            t.player_message,
            t.context
        FROM annotations a
        JOIN tasks t ON a.task_id = t.id
        '''
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        return df
    
    def calculate_fleiss_kappa(self, annotations_df: pd.DataFrame, 
                               task_ids: List[int] = None) -> Dict:
        """
        Tính Fleiss' Kappa cho categorical variables
        Dựa trên "The Measurement of Observer Agreement for Categorical Data" (Fleiss, 1971)
        """
        
        if task_ids is None:
            # Lấy các tasks có ít nhất 3 annotations (cho pilot study)
            task_counts = annotations_df['task_id'].value_counts()
            task_ids = task_counts[task_counts >= 3].index.tolist()
        
        results = {}
        
        # 1. Tính cho NPC State
        state_kappa = self._calculate_fleiss_for_category(
            annotations_df, task_ids, 'npc_state'
        )
        results['npc_state'] = {
            'fleiss_kappa': state_kappa,
            'interpretation': self._interpret_kappa(state_kappa)
        }
        
        # 2. Tính cho Emotional Intensity
        emotion_kappa = self._calculate_fleiss_for_category(
            annotations_df, task_ids, 'emotional_intensity'
        )
        results['emotional_intensity'] = {
            'fleiss_kappa': emotion_kappa,
            'interpretation': self._interpret_kappa(emotion_kappa)
        }
        
        return results
    
    def _calculate_fleiss_for_category(self, df: pd.DataFrame, 
                                       task_ids: List[int], 
                                       category: str) -> float:
        """Tính Fleiss' Kappa cho một category cụ thể"""
        
        # Tạo rating matrix
        categories = sorted(df[category].unique())
        n_categories = len(categories)
        
        # Lọc tasks có multiple annotations
        multi_annot_tasks = []
        for task_id in task_ids:
            task_df = df[df['task_id'] == task_id]
            if len(task_df) >= 2:  # Ít nhất 2 annotations
                multi_annot_tasks.append(task_id)
        
        n_items = len(multi_annot_tasks)
        if n_items == 0:
            return 0.0
        
        # Khởi tạo rating matrix
        rating_matrix = np.zeros((n_items, n_categories))
        
        # Điền matrix
        for i, task_id in enumerate(multi_annot_tasks):
            task_df = df[df['task_id'] == task_id]
            for annot_idx, row in task_df.iterrows():
                cat_value = row[category]
                cat_index = categories.index(cat_value)
                rating_matrix[i, cat_index] += 1
        
        # Tính Fleiss' Kappa
        n_annotators = rating_matrix.sum(axis=1).mean()
        
        # Tính Pj (proportion of assignments to category j)
        Pj = rating_matrix.sum(axis=0) / (n_items * n_annotators)
        
        # Tính Pi (proportion of agreeing pairs for item i)
        Pi = ((rating_matrix ** 2).sum(axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
        
        # Tính Pbar (mean of Pi)
        Pbar = Pi.mean()
        
        # Tính Pbar_e (expected agreement by chance)
        Pbar_e = (Pj ** 2).sum()
        
        # Tính Kappa
        if Pbar_e == 1:
            kappa = 1.0
        else:
            kappa = (Pbar - Pbar_e) / (1 - Pbar_e)
        
        return kappa
    
    def calculate_krippendorff_alpha(self, annotations_df: pd.DataFrame) -> Dict:
        """
        Tính Krippendorff's Alpha - robust hơn cho small samples
        Dựa trên "Content Analysis: An Introduction to Its Methodology" (Krippendorff, 2018)
        """
        
        # Đơn giản hóa: sử dụng scipy's kendall tau cho ordinal data
        results = {}
        
        # Tính cho các cặp annotators
        annotator_pairs = self._get_annotator_pairs(annotations_df)
        
        alpha_scores = []
        for ann1, ann2 in annotator_pairs:
            # Lấy các tasks mà cả 2 cùng annotate
            common_tasks = self._get_common_tasks(annotations_df, ann1, ann2)
            
            if len(common_tasks) >= 2:
                # Tính agreement cho từng category
                for category in ['npc_state', 'emotional_intensity']:
                    values1 = []
                    values2 = []
                    
                    for task_id in common_tasks:
                        val1 = annotations_df[
                            (annotations_df['task_id'] == task_id) & 
                            (annotations_df['annotator_id'] == ann1)
                        ][category].iloc[0]
                        
                        val2 = annotations_df[
                            (annotations_df['task_id'] == task_id) & 
                            (annotations_df['annotator_id'] == ann2)
                        ][category].iloc[0]
                        
                        values1.append(val1)
                        values2.append(val2)
                    
                    # Tính agreement
                    if category == 'emotional_intensity':  # Ordinal
                        tau, _ = stats.kendalltau(values1, values2)
                        alpha_scores.append(tau)
                    else:  # Nominal
                        agreement = sum(v1 == v2 for v1, v2 in zip(values1, values2)) / len(values1)
                        alpha_scores.append(agreement)
        
        if alpha_scores:
            avg_alpha = np.mean(alpha_scores)
        else:
            avg_alpha = 0.0
        
        results['krippendorff_alpha'] = {
            'value': avg_alpha,
            'interpretation': self._interpret_alpha(avg_alpha),
            'n_pairs': len(annotator_pairs)
        }
        
        return results
    
    def _get_annotator_pairs(self, df: pd.DataFrame) -> List[Tuple]:
        """Lấy tất cả cặp annotators"""
        annotators = df['annotator_id'].unique()
        return list(itertools.combinations(annotators, 2))
    
    def _get_common_tasks(self, df: pd.DataFrame, 
                         annotator1: int, 
                         annotator2: int) -> List[int]:
        """Lấy tasks mà cả 2 annotators cùng annotate"""
        tasks1 = set(df[df['annotator_id'] == annotator1]['task_id'])
        tasks2 = set(df[df['annotator_id'] == annotator2]['task_id'])
        return list(tasks1.intersection(tasks2))
    
    def calculate_semantic_similarity(self, annotations_df: pd.DataFrame) -> Dict:
        """
        Tính semantic similarity giữa các annotations sử dụng TF-IDF cosine similarity
        Dựa trên "Semantic Similarity for Text Quality Assessment" (2022)
        """
        
        # Group by task
        task_groups = annotations_df.groupby('task_id')
        
        similarities = []
        
        for task_id, group in task_groups:
            if len(group) >= 2:
                responses = group['npc_response'].tolist()
                
                # Tính TF-IDF vectors
                vectorizer = TfidfVectorizer()
                try:
                    tfidf_matrix = vectorizer.fit_transform(responses)
                    
                    # Tính pairwise cosine similarities
                    cosine_sim = cosine_similarity(tfidf_matrix)
                    
                    # Lấy upper triangle (không tính diagonal)
                    n = len(responses)
                    for i in range(n):
                        for j in range(i+1, n):
                            similarities.append(cosine_sim[i, j])
                except:
                    # Nếu không có từ nào chung
                    continue
        
        if similarities:
            avg_similarity = np.mean(similarities)
            std_similarity = np.std(similarities)
        else:
            avg_similarity = 0.0
            std_similarity = 0.0
        
        return {
            'semantic_similarity': {
                'mean': avg_similarity,
                'std': std_similarity,
                'n_comparisons': len(similarities),
                'interpretation': self._interpret_similarity(avg_similarity)
            }
        }
    
    def _interpret_kappa(self, kappa: float) -> str:
        """Interpret Fleiss' Kappa theo Landis & Koch (1977)"""
        if kappa < 0:
            return "Poor agreement"
        elif kappa <= 0.2:
            return "Slight agreement"
        elif kappa <= 0.4:
            return "Fair agreement"
        elif kappa <= 0.6:
            return "Moderate agreement"
        elif kappa <= 0.8:
            return "Substantial agreement"
        else:
            return "Almost perfect agreement"
    
    def _interpret_alpha(self, alpha: float) -> str:
        """Interpret Krippendorff's Alpha"""
        if alpha < 0.667:
            return "Unreliable"
        elif alpha < 0.8:
            return "Marginally reliable"
        elif alpha < 0.9:
            return "Reliable"
        else:
            return "Highly reliable"
    
    def _interpret_similarity(self, similarity: float) -> str:
        """Interpret semantic similarity"""
        if similarity < 0.3:
            return "Low similarity - high diversity"
        elif similarity < 0.6:
            return "Moderate similarity"
        elif similarity < 0.8:
            return "High similarity"
        else:
            return "Very high similarity - possible copying"
    
    def generate_iaa_report(self, output_path="iaa_report.md"):
        """Tạo comprehensive IAA report"""
        
        print("📊 Đang tính Inter-Annotator Agreement...")
        
        # Load annotations
        df = self.load_annotations()
        
        if len(df) == 0:
            print("⚠️ Không có annotations để phân tích")
            return
        
        # Tính các metrics
        fleiss_results = self.calculate_fleiss_kappa(df)
        alpha_results = self.calculate_krippendorff_alpha(df)
        similarity_results = self.calculate_semantic_similarity(df)
        
        # Tạo report
        report = f"""# INTER-ANNOTATOR AGREEMENT REPORT

## 📈 Tổng quan

**Ngày tạo:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}
**Tổng số annotations:** {len(df)}
**Số annotators:** {df['annotator_id'].nunique()}
**Số tasks:** {df['task_id'].nunique()}

## 📊 Agreement Statistics

### 1. Fleiss' Kappa
| Category | Kappa | Interpretation |
|----------|-------|----------------|
| NPC State | {fleiss_results['npc_state']['fleiss_kappa']:.3f} | {fleiss_results['npc_state']['interpretation']} |
| Emotional Intensity | {fleiss_results['emotional_intensity']['fleiss_kappa']:.3f} | {fleiss_results['emotional_intensity']['interpretation']} |

*Ghi chú: Kappa > 0.6 được coi là acceptable cho nghiên cứu (Landis & Koch, 1977)*

### 2. Krippendorff's Alpha
**Alpha:** {alpha_results['krippendorff_alpha']['value']:.3f}
**Interpretation:** {alpha_results['krippendorff_alpha']['interpretation']}
**Số cặp annotators:** {alpha_results['krippendorff_alpha']['n_pairs']}

*Ghi chú: Alpha > 0.8 được coi là reliable (Krippendorff, 2018)*

### 3. Semantic Similarity
**Mean similarity:** {similarity_results['semantic_similarity']['mean']:.3f}
**Standard deviation:** {similarity_results['semantic_similarity']['std']:.3f}
**Interpretation:** {similarity_results['semantic_similarity']['interpretation']}
**Số so sánh:** {similarity_results['semantic_similarity']['n_comparisons']}

## 📈 Phân bố annotations

### Phân bố theo NPC State:

In [ ]:

### Chất lượng trung bình:
#- Naturalness: {df['quality_naturalness'].mean():.2f}/5
#- Consistency: {df['quality_consistency'].mean():.2f}/5  
#- Appropriateness: {df['quality_appropriateness'].mean():.2f}/5

### Confidence scores:
#- Mean confidence: {df['confidence_score'].mean():.2f}/5

## 📊 Per-Annotator Statistics

"""
        
        # Thêm statistics theo từng annotator
        annotator_stats = df.groupby('annotator_id').agg({
            'quality_naturalness': 'mean',
            'quality_consistency': 'mean',
            'quality_appropriateness': 'mean',
            'confidence_score': 'mean',
            'task_id': 'count'
        }).rename(columns={'task_id': 'annotation_count'})
        
        report += annotator_stats.to_markdown()
        
        report += """

## 🎯 Recommendations

### Nếu Kappa < 0.6:
1. **Tổ chức training session lại** với examples rõ ràng hơn
2. **Clarify annotation guidelines** cho các category gây confusion
3. **Thêm more examples** cho các edge cases
4. **Consider merging categories** nếu quá ambiguous

### Nếu Semantic Similarity quá cao (>0.8):
1. **Kiểm tra copying** giữa các annotators
2. **Thêm response diversity requirements**
3. **Khuyến khích creativity** trong guidelines

### Nếu Semantic Similarity quá thấp (<0.3):
1. **Đảm bảo character consistency** được giữ vững
2. **Review guidelines** để đảm bảo clear expectations
3. **Thêm constraints** để giữ response trong phạm vi chấp nhận được

### Nếu Annotation Time quá ngắn (<30s):
1. **Kiểm tra quality** của annotations
2. **Đảm bảo annotators không rush**
3. **Thêm minimum time requirements**

## 📊 Visualizations (xem file plots/)

Các biểu đồ đã được lưu trong thư mục `plots/`:
1. `state_distribution.png` - Phân bố NPC states
2. `emotion_distribution.png` - Phân bố emotional intensity
3. `quality_scores.png` - Phân bố quality scores
4. `annotator_agreement.png` - Agreement heatmap
5. `annotation_time_distribution.png` - Phân bố thời gian annotation

## 🚀 Next Steps

1. **Nếu IAA acceptable (>0.6):** Tiến hành annotation full dataset
2. **Nếu IAA marginal (0.4-0.6):** Điều chỉnh guidelines và chạy pilot lại
3. **Nếu IAA poor (<0.4):** Xem xét lại toàn bộ annotation framework

### Action Plan:
- [ ] Review IAA scores với team
- [ ] Identify problematic categories
- [ ] Update guidelines nếu cần
- [ ] Retrain annotators nếu cần
- [ ] Run another pilot nếu scores thấp

---
*Report generated với methodology từ:*
*- Fleiss, J. L. (1971). "Measuring nominal scale agreement among many raters"*
*- Krippendorff, K. (2018). "Content Analysis: An Introduction to Its Methodology"*
*- Landis, J. R., & Koch, G. G. (1977). "The measurement of observer agreement"*
"""
        
        # Lưu report
        os.makedirs(os.path.dirname(output_path) if os.path.dirname(output_path) else '.', exist_ok=True)
        
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(report)
        
        # Tạo visualizations
        self.create_visualizations(df)
        
        # Lưu detailed results
        self.save_detailed_results(df, fleiss_results, alpha_results, similarity_results, cohen_results)
        
        print(f"✅ Đã tạo IAA report: {output_path}")
        
        return {
            'fleiss': fleiss_results,
            'alpha': alpha_results,
            'similarity': similarity_results,
            'cohen': cohen_results,
            'time_stats': time_stats
        }
    
    def create_visualizations(self, df: pd.DataFrame):
        """Tạo visualizations cho IAA analysis"""
        
        import matplotlib
        matplotlib.use('Agg')  # For headless environments
        
        os.makedirs("plots", exist_ok=True)
        
        # 1. NPC State Distribution
        plt.figure(figsize=(10, 6))
        state_counts = df['npc_state'].value_counts()
        colors = ['#4CAF50', '#FFC107', '#F44336', '#2196F3']  # green, yellow, red, blue
        
        # Đảm bảo tất cả states đều có trong biểu đồ
        all_states = ['normal', 'alert', 'combat', 'injured']
        for state in all_states:
            if state not in state_counts.index:
                state_counts[state] = 0
        
        state_counts = state_counts.reindex(all_states)
        
        bars = plt.bar(state_counts.index, state_counts.values, color=colors)
        
        plt.title('Phân bố NPC States', fontsize=16, fontweight='bold')
        plt.xlabel('NPC State', fontsize=12)
        plt.ylabel('Số annotations', fontsize=12)
        
        # Thêm số trên mỗi bar
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.savefig('plots/state_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2. Emotional Intensity Distribution
        plt.figure(figsize=(10, 6))
        emotion_counts = df['emotional_intensity'].value_counts().sort_index()
        
        # Đảm bảo tất cả levels đều có
        for level in range(1, 6):
            if level not in emotion_counts.index:
                emotion_counts[level] = 0
        
        emotion_counts = emotion_counts.sort_index()
        
        # Tạo gradient color từ xanh (calm) đến đỏ (intense)
        colors = plt.cm.RdYlGn_r(np.linspace(0, 1, len(emotion_counts)))
        
        bars = plt.bar(emotion_counts.index.astype(str), emotion_counts.values, color=colors)
        
        plt.title('Phân bố Emotional Intensity', fontsize=16, fontweight='bold')
        plt.xlabel('Emotional Intensity Level', fontsize=12)
        plt.ylabel('Số annotations', fontsize=12)
        
        # Thêm mô tả cho từng level
        emotion_labels = {
            '1': 'Neutral\n(1)',
            '2': 'Mild\n(2)', 
            '3': 'Medium\n(3)',
            '4': 'Strong\n(4)',
            '5': 'Very Strong\n(5)'
        }
        
        plt.xticks(list(emotion_counts.index.astype(str)), 
                  [emotion_labels[str(i)] for i in emotion_counts.index])
        
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.savefig('plots/emotion_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 3. Quality Scores Distribution
        plt.figure(figsize=(12, 8))
        
        quality_metrics = ['quality_naturalness', 'quality_consistency', 'quality_appropriateness']
        metric_labels = ['Naturalness', 'Consistency', 'Appropriateness']
        
        data = [df[metric] for metric in quality_metrics]
        
        box = plt.boxplot(data, labels=metric_labels, patch_artist=True)
        
        # Màu cho các box
        colors = ['#FF9999', '#99FF99', '#9999FF']
        for patch, color in zip(box['boxes'], colors):
            patch.set_facecolor(color)
        
        plt.title('Phân bố Quality Scores', fontsize=16, fontweight='bold')
        plt.ylabel('Score (1-5)', fontsize=12)
        plt.ylim(0.5, 5.5)
        
        # Thêm mean values
        for i, metric in enumerate(quality_metrics, 1):
            mean_val = df[metric].mean()
            plt.text(i, 5.2, f'Mean: {mean_val:.2f}', 
                    ha='center', va='bottom', fontsize=10)
        
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('plots/quality_scores.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 4. Annotation Time Distribution (nếu có)
        if 'annotation_time_seconds' in df.columns and not df['annotation_time_seconds'].isna().all():
            plt.figure(figsize=(10, 6))
            
            times = df['annotation_time_seconds'].dropna()
            
            plt.hist(times, bins=30, edgecolor='black', alpha=0.7)
            plt.axvline(times.mean(), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {times.mean():.1f}s')
            plt.axvline(times.median(), color='green', linestyle='dashed', linewidth=2, label=f'Median: {times.median():.1f}s')
            
            plt.title('Phân bố Thời gian Annotation', fontsize=16, fontweight='bold')
            plt.xlabel('Thời gian (giây)', fontsize=12)
            plt.ylabel('Số annotations', fontsize=12)
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig('plots/annotation_time_distribution.png', dpi=300, bbox_inches='tight')
            plt.close()
        
        print("✅ Đã tạo visualizations trong thư mục plots/")
    
    def save_detailed_results(self, df: pd.DataFrame, fleiss_results: Dict, 
                             alpha_results: Dict, similarity_results: Dict,
                             cohen_results: Dict):
        """Lưu detailed results ra JSON file"""
        
        detailed_results = {
            'summary': {
                'total_annotations': len(df),
                'n_annotators': df['annotator_id'].nunique(),
                'n_tasks': df['task_id'].nunique(),
                'date_generated': datetime.now().isoformat()
            },
            'fleiss_kappa': fleiss_results,
            'krippendorff_alpha': alpha_results,
            'semantic_similarity': similarity_results,
            'cohen_kappa': cohen_results,
            'statistics': {
                'state_distribution': df['npc_state'].value_counts().to_dict(),
                'emotion_distribution': df['emotional_intensity'].value_counts().sort_index().to_dict(),
                'quality_scores': {
                    'naturalness_mean': float(df['quality_naturalness'].mean()),
                    'consistency_mean': float(df['quality_consistency'].mean()),
                    'appropriateness_mean': float(df['quality_appropriateness'].mean())
                }
            }
        }
        
        with open('plots/detailed_results.json', 'w', encoding='utf-8') as f:
            json.dump(detailed_results, f, ensure_ascii=False, indent=2)
        
        print("✅ Đã lưu detailed results: plots/detailed_results.json")

class AnnotationQualityController:
    """Quality control pipeline dựa trên research-based methods"""
    
    def __init__(self):
        self.quality_thresholds = {
            'min_response_length': 3,  # Ít nhất 3 từ
            'max_response_length': 50, # Tối đa 50 từ
            'min_quality_score': 3,    # Chất lượng tối thiểu
            'max_similarity_with_others': 0.9,  # Không copy quá nhiều
            'required_fields': ['npc_response', 'npc_state', 'emotional_intensity']
        }
        
        self.modern_words = ['ok', 'hello', 'hi', 'yes', 'no', 'sorry', 'thanks', 
                           'okay', 'hey', 'please', 'thank you', 'thanks', 'cool',
                           'awesome', 'amazing', 'wow', 'lol', 'haha', 'omg']
        
    def validate_annotations(self, annotations_df: pd.DataFrame) -> Dict:
        """Validate annotations theo multiple criteria"""
        
        validation_results = {
            'total_annotations': len(annotations_df),
            'passed_validation': 0,
            'failed_validation': 0,
            'failure_reasons': defaultdict(int),
            'failed_samples': [],
            'warnings': []
        }
        
        for idx, row in annotations_df.iterrows():
            is_valid, reason, warnings = self._validate_single_annotation(row)
            
            if is_valid:
                validation_results['passed_validation'] += 1
            else:
                validation_results['failed_validation'] += 1
                validation_results['failure_reasons'][reason] += 1
                validation_results['failed_samples'].append({
                    'task_id': int(row['task_id']),
                    'annotator_id': int(row['annotator_id']),
                    'reason': reason,
                    'response': str(row['npc_response'])[:100] + '...' if len(str(row['npc_response'])) > 100 else str(row['npc_response'])
                })
            
            # Thêm warnings
            for warning in warnings:
                validation_results['warnings'].append({
                    'task_id': int(row['task_id']),
                    'annotator_id': int(row['annotator_id']),
                    'warning': warning
                })
        
        # Tính pass rate
        if validation_results['total_annotations'] > 0:
            validation_results['pass_rate'] = (
                validation_results['passed_validation'] / 
                validation_results['total_annotations'] * 100
            )
        else:
            validation_results['pass_rate'] = 0
        
        # Thống kê warnings
        warning_counts = defaultdict(int)
        for warning in validation_results['warnings']:
            warning_counts[warning['warning']] += 1
        validation_results['warning_counts'] = dict(warning_counts)
        
        return validation_results
    
    def _validate_single_annotation(self, annotation: pd.Series) -> Tuple[bool, str, List[str]]:
        """Validate single annotation"""
        
        warnings = []
        
        # 1. Check required fields
        for field in self.quality_thresholds['required_fields']:
            if field not in annotation or pd.isna(annotation[field]) or str(annotation[field]) == '':
                return False, f'missing_{field}', warnings
        
        # 2. Check response length
        response = str(annotation['npc_response'])
        word_count = len(response.split())
        
        if word_count < self.quality_thresholds['min_response_length']:
            return False, 'response_too_short', warnings
        
        if word_count > self.quality_thresholds['max_response_length']:
            warnings.append('response_too_long')
        
        # 3. Check quality scores
        quality_fields = ['quality_naturalness', 'quality_consistency', 'quality_appropriateness']
        for field in quality_fields:
            if field in annotation and pd.notna(annotation[field]):
                if annotation[field] < self.quality_thresholds['min_quality_score']:
                    warnings.append(f'low_{field}')
        
        # 4. Check confidence score
        if 'confidence_score' in annotation and pd.notna(annotation['confidence_score']):
            if annotation['confidence_score'] < 3:
                warnings.append('low_confidence')
        
        # 5. Check for modern language/anachronisms
        response_lower = response.lower()
        modern_words_found = [word for word in self.modern_words if word in response_lower]
        if modern_words_found:
            warnings.append(f'anachronism: {", ".join(modern_words_found[:3])}')
        
        # 6. Check for inappropriate content
        inappropriate_terms = ['fuck', 'shit', 'damn', 'hell', 'bastard', 'asshole']
        inappropriate_found = [term for term in inappropriate_terms if term in response_lower]
        if inappropriate_found:
            warnings.append(f'inappropriate_language: {", ".join(inappropriate_found[:3])}')
        
        return True, 'passed', warnings
    
    def detect_pattern_repetition(self, annotations_df: pd.DataFrame, 
                                 similarity_threshold: float = 0.8) -> Dict:
        """Phát hiện pattern repetition giữa các annotators"""
        
        results = {
            'high_similarity_pairs': [],
            'possible_copied_responses': [],
            'unique_annotators': annotations_df['annotator_id'].nunique()
        }
        
        # Group by task
        task_groups = annotations_df.groupby('task_id')
        
        for task_id, group in task_groups:
            if len(group) >= 2:
                responses = group['npc_response'].tolist()
                annotators = group['annotator_id'].tolist()
                
                # Tính similarity matrix
                vectorizer = TfidfVectorizer()
                try:
                    tfidf_matrix = vectorizer.fit_transform(responses)
                    cosine_sim = cosine_similarity(tfidf_matrix)
                    
                    # Kiểm tra các cặp có similarity cao
                    n = len(responses)
                    for i in range(n):
                        for j in range(i+1, n):
                            if cosine_sim[i, j] > similarity_threshold:
                                results['high_similarity_pairs'].append({
                                    'task_id': int(task_id),
                                    'annotator1': int(annotators[i]),
                                    'annotator2': int(annotators[j]),
                                    'similarity': float(cosine_sim[i, j]),
                                    'response1': responses[i][:100] + '...' if len(responses[i]) > 100 else responses[i],
                                    'response2': responses[j][:100] + '...' if len(responses[j]) > 100 else responses[j]
                                })
                                
                                # Nếu similarity rất cao, có thể là copying
                                if cosine_sim[i, j] > 0.95:
                                    results['possible_copied_responses'].append({
                                        'task_id': int(task_id),
                                        'annotator_pair': (int(annotators[i]), int(annotators[j])),
                                        'similarity': float(cosine_sim[i, j])
                                    })
                except:
                    continue
        
        return results
    
    def calculate_annotator_reliability(self, annotations_df: pd.DataFrame) -> Dict:
        """Tính reliability score cho từng annotator"""
        
        if len(annotations_df) == 0:
            return {}
        
        annotator_stats = annotations_df.groupby('annotator_id').agg({
            'quality_naturalness': ['mean', 'std', 'count'],
            'quality_consistency': ['mean', 'std', 'count'],
            'quality_appropriateness': ['mean', 'std', 'count'],
            'confidence_score': 'mean'
        })
        
        # Flatten column names
        annotator_stats.columns = ['_'.join(col).strip() for col in annotator_stats.columns.values]
        
        # Tính reliability score
        reliability_scores = {}
        for annotator_id in annotator_stats.index:
            stats = annotator_stats.loc[annotator_id]
            
            # Tính score dựa trên chất lượng và consistency
            quality_score = (
                stats.get('quality_naturalness_mean', 3) +
                stats.get('quality_consistency_mean', 3) +
                stats.get('quality_appropriateness_mean', 3)
            ) / 3
            
            # Tính variability (thấp hơn tốt hơn)
            variability = np.mean([
                stats.get('quality_naturalness_std', 0),
                stats.get('quality_consistency_std', 0),
                stats.get('quality_appropriateness_std', 0)
            ])
            
            # Tính reliability score (0-1)
            reliability = quality_score / 5 * (1 - min(variability / 2, 0.5))
            
            reliability_scores[annotator_id] = {
                'reliability_score': float(reliability),
                'quality_score': float(quality_score),
                'variability': float(variability),
                'annotation_count': int(stats.get('quality_naturalness_count', 0)),
                'confidence_mean': float(stats.get('confidence_score_mean', 3))
            }
        
        return reliability_scores
    
    def generate_quality_report(self, validation_results: Dict, 
                               pattern_results: Dict = None,
                               reliability_scores: Dict = None,
                               output_path="quality_report.md"):
        """Tạo comprehensive quality report"""
        
        report = f"""# ANNOTATION QUALITY CONTROL REPORT

## 📊 Tổng quan chất lượng

**Tổng số annotations:** {validation_results['total_annotations']}
**Annotations passed:** {validation_results['passed_validation']}
**Annotations failed:** {validation_results['failed_validation']}
**Pass rate:** {validation_results['pass_rate']:.1f}%

## 🚨 Failure Analysis

### Lý do failed (tổng số):
"""
        
        for reason, count in validation_results['failure_reasons'].items():
            report += f"- **{reason}:** {count} annotations\n"
        
        report += f"""

## ⚠️ Warnings Analysis

### Cảnh báo phát hiện:
"""
        
        if validation_results.get('warning_counts'):
            for warning, count in validation_results['warning_counts'].items():
                report += f"- **{warning}:** {count} annotations\n"
        else:
            report += "Không có warnings.\n"
        
        report += f"""

## 🔍 Pattern Repetition Analysis
"""
        
        if pattern_results:
            report += f"""
**Số annotators:** {pattern_results['unique_annotators']}
**Số cặp có similarity > 0.8:** {len(pattern_results['high_similarity_pairs'])}
**Số cặp có thể copied (similarity > 0.95):** {len(pattern_results['possible_copied_responses'])}

### Các cặp có similarity cao nhất (top 5):
"""
            
            # Sắp xếp theo similarity
            high_similarity = sorted(pattern_results['high_similarity_pairs'], 
                                   key=lambda x: x['similarity'], reverse=True)[:5]
            
            for i, pair in enumerate(high_similarity, 1):
                report += f"""
{i}. **Task {pair['task_id']}** (Annotators: {pair['annotator1']} & {pair['annotator2']})
   Similarity: {pair['similarity']:.3f}
   Response 1: {pair['response1']}
   Response 2: {pair['response2']}
"""
        
        report += f"""

## 👤 Annotator Reliability Scores
"""
        
        if reliability_scores:
            report += """
| Annotator ID | Reliability Score | Quality Score | Variability | Count |
|--------------|-------------------|---------------|-------------|-------|
"""
            
            for annotator_id, scores in sorted(reliability_scores.items(), 
                                             key=lambda x: x[1]['reliability_score'], 
                                             reverse=True):
                report += f"| {annotator_id} | {scores['reliability_score']:.3f} | {scores['quality_score']:.2f} | {scores['variability']:.2f} | {scores['annotation_count']} |\n"
            
            # Thêm interpretation
            report += f"""

### Interpretation:
- **Reliability Score > 0.8:** Excellent annotator
- **Reliability Score 0.6-0.8:** Good annotator
- **Reliability Score 0.4-0.6:** Needs improvement
- **Reliability Score < 0.4:** Consider retraining
"""
        
        report += f"""

## 📝 Failed Samples (first 10)

"""
        
        if validation_results['failed_samples']:
            for i, sample in enumerate(validation_results['failed_samples'][:10]):
                report += f"{i+1}. **Task {sample['task_id']}** (Annotator {sample['annotator_id']})\n"
                report += f"   Reason: {sample['reason']}\n"
                report += f"   Response: {sample['response']}\n\n"
        else:
            report += "Không có failed samples.\n"
        
        report += f"""

## 🎯 Recommendations

### Nếu pass rate > 90%:
✅ **Chất lượng tốt** - Có thể tiếp tục với annotation scale-up

### Nếu pass rate 70-90%:
⚠️ **Cần cải thiện** - Tổ chức additional training cho annotators

### Nếu pass rate < 70%:
❌ **Vấn đề nghiêm trọng** - Cần xem xét lại toàn bộ annotation process

### Nếu có nhiều high similarity pairs (>20%):
⚠️ **Có thể có copying** - Cần xem xét monitoring và guidelines

### Action Items:
1. Review failed samples để hiểu patterns
2. Address common failure reasons
3. Retrain annotators có reliability score thấp
4. Update guidelines dựa trên findings
5. Consider adding more validation rules

---
*Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}*
"""
        
        # Lưu report
        os.makedirs(os.path.dirname(output_path) if os.path.dirname(output_path) else '.', exist_ok=True)
        
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(report)
        
        print(f"✅ Đã tạo quality report: {output_path}")
        
        return report

# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    print("🔍 Bắt đầu Quality Control Pipeline...")
    print("="*60)
    
    # 1. Tính Inter-Annotator Agreement
    print("\n1. 📊 Tính Inter-Annotator Agreement...")
    iaa = InterAnnotatorAgreement()
    
    try:
        iaa_report = iaa.generate_iaa_report()
        if iaa_report:
            print("✅ IAA analysis completed")
    except Exception as e:
        print(f"⚠️ Lỗi trong IAA analysis: {e}")
        print("Tạo database mẫu để test...")
        # Tạo database mẫu để test
        create_sample_database()
        iaa_report = iaa.generate_iaa_report()
    
    # 2. Quality Validation
    print("\n2. 🎯 Validating annotation quality...")
    df = iaa.load_annotations()
    
    if len(df) > 0:
        qc = AnnotationQualityController()
        
        # Validate annotations
        validation_results = qc.validate_annotations(df)
        
        # Detect pattern repetition
        pattern_results = qc.detect_pattern_repetition(df)
        
        # Calculate reliability scores
        reliability_scores = qc.calculate_annotator_reliability(df)
        
        # Generate quality report
        qc_report = qc.generate_quality_report(
            validation_results, 
            pattern_results, 
            reliability_scores
        )
        
        print(f"✅ Quality validation completed.")
        print(f"   Pass rate: {validation_results['pass_rate']:.1f}%")
        print(f"   Failed: {validation_results['failed_validation']} annotations")
        
        if reliability_scores:
            avg_reliability = np.mean([s['reliability_score'] for s in reliability_scores.values()])
            print(f"   Avg reliability score: {avg_reliability:.3f}")
    
    print("\n" + "="*60)
    print("🎉 QUALITY CONTROL PIPELINE HOÀN THÀNH!")
    print("="*60)
    print("\n📁 Output files:")
    print("  - iaa_report.md (Inter-Annotator Agreement analysis)")
    print("  - quality_report.md (Quality validation results)")
    print("  - plots/ (Visualizations)")
    print("  - plots/detailed_results.json (Detailed metrics)")
    print("\n📊 Key metrics to check:")
    print("  1. Fleiss' Kappa > 0.6 (Moderate agreement)")
    print("  2. Pass rate > 85%")
    print("  3. Avg reliability score > 0.7")
    print("\n🚀 Next steps:")
    print("  1. Review reports với team")
    print("  2. Address quality issues")
    print("  3. Retrain annotators nếu cần")
    print("  4. Update annotation guidelines")
    print("  5. Proceed with full-scale annotation")

def create_sample_database():
    """Tạo database mẫu để test nếu không có data"""
    print("📝 Tạo database mẫu với 60 annotations từ 3 annotators...")
    
    conn = sqlite3.connect('annotations.db')
    cursor = conn.cursor()
    
    # Tạo tables nếu chưa có
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            player_message TEXT NOT NULL,
            context TEXT,
            npc_state TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            assigned_to INTEGER,
            completed BOOLEAN DEFAULT FALSE
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS annotations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            task_id INTEGER NOT NULL,
            annotator_id INTEGER NOT NULL,
            npc_response TEXT NOT NULL,
            npc_state TEXT NOT NULL,
            emotional_intensity INTEGER,
            dialogue_acts TEXT,
            quality_naturalness INTEGER,
            quality_consistency INTEGER,
            quality_appropriateness INTEGER,
            confidence_score INTEGER,
            annotation_time_seconds INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Tạo sample tasks
    sample_tasks = [
        ("Xin chào", "Ban ngày, cổng thành", "normal"),
        ("Cho tôi qua đi", "Player tiến gần cổng", "alert"),
        ("Ta sẽ giết ngươi!", "", "combat"),
        ("Ngươi thua rồi", "NPC bị thương nặng", "injured"),
        ("Trời hôm nay đẹp nhỉ", "Buổi sáng yên tĩnh", "normal"),
        ("Có chuyện gì thế?", "Player bị cảnh báo", "alert"),
        ("(Tấn công)", "Trận chiến bắt đầu", "combat"),
        ("Có cần giúp không?", "NPC bị thương", "injured"),
    ]
    
    task_ids = []
    for msg, ctx, state in sample_tasks:
        cursor.execute('INSERT INTO tasks (player_message, context, npc_state) VALUES (?, ?, ?)',
                      (msg, ctx, state))
        task_ids.append(cursor.lastrowid)
    
    # Tạo sample annotations (3 annotators cho mỗi task)
    sample_responses = {
        "normal": [
            "Chào công dân. Giữ trật tự và di chuyển đi.",
            "Chào. Đừng gây rắc rối ở đây.",
            "Chào. Hãy tuân thủ luật lệ."
        ],
        "alert": [
            "Dừng lại! Không được bước thêm bước nào nữa.",
            "Đứng yên! Tay để xa vũ khí.",
            "Cảnh báo! Dừng lại ngay lập tức."
        ],
        "combat": [
            "Chết đi, kẻ xâm nhập!",
            "Ta sẽ nghiền nát ngươi!",
            "Ngươi không qua được đâu!"
        ],
        "injured": [
            "Làm... làm ơn... tha cho tôi...",
            "Tôi... tôi đầu hàng...",
            "Xin đừng... tôi còn gia đình..."
        ]
    }
    
    for task_id in task_ids:
        # Lấy state của task
        cursor.execute('SELECT npc_state FROM tasks WHERE id = ?', (task_id,))
        state = cursor.fetchone()[0]
        
        # Tạo 3 annotations cho mỗi task
        for annotator_id in range(1, 4):
            response = sample_responses[state][annotator_id - 1]
            emotional_intensity = {"normal": 2, "alert": 4, "combat": 5, "injured": 4}[state]
            
            cursor.execute('''
                INSERT INTO annotations 
                (task_id, annotator_id, npc_response, npc_state, emotional_intensity,
                 quality_naturalness, quality_consistency, quality_appropriateness,
                 confidence_score, annotation_time_seconds)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                task_id, annotator_id, response, state, emotional_intensity,
                4, 4, 4, 4, np.random.randint(30, 120)
            ))
    
    conn.commit()
    conn.close()
    
    print("✅ Đã tạo database mẫu với 24 annotations")